In [1]:
import os
import time
import matplotlib.pyplot as plt
import random
from pennylane import numpy as np
from tqdm import tqdm
#from qbmqsp.qbm import QBM
from qbmqsp.utils import construct_multi_fcqbm_pauli_strings
from gen_data import xxz_gibbs_state, basis_encoding, gen_boltzmann_dist, gen_discrete_gauss_dist
from qbmqsp.src.utils import import_dataset, split_dataset_labels, split_data
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score)
import scipy.linalg as spl

import pennylane as qml
from pennylane import numpy as np
from pennylane.pauli.utils import string_to_pauli_word

from qbmqsp.hamiltonian import Hamiltonian
from qbmqsp.qsp_phase_engine import QSPPhaseEngine
from qbmqsp.qevt import QEVT
from qbmqsp.rel_ent import relative_entropy

import seaborn as sns
import numpy


import itertools

from functools import partial

In [2]:
def generate_pauli_strings_tfim(num_qubits,n_visible,restricted=False,transverse=True):
    """
    Generate Pauli strings for a transverse field Ising model as a 
    boltzmann machine .
    
    Parameters:
    num_qubits (int): Number of qubits in the quantum Boltzmann machine.
    n_visible (int): Number if visible units.
    restricted (bool) 
    
    Returns:
    list: List of Pauli strings representing the Hamiltonian.
    """
    pauli_strings = []

    # Local transverse field terms (X_i)
    
    if transverse==True:
        for i in range(num_qubits):
            
            pauli_string = ['I'] * num_qubits
            pauli_string[i] = 'X'
            
            pauli_strings.append(''.join(pauli_string))

    for i in range(num_qubits):
        
        pauli_string = ['I'] * num_qubits
        pauli_string[i] = 'Z'
        
        pauli_strings.append(''.join(pauli_string))

    # Interaction terms (Z_i Z_j)
    
    
    for i, j in itertools.combinations(range(num_qubits), 2):
        if restricted:
            if i<n_visible and j>=n_visible:
                pauli_string = ['I'] * num_qubits
    
                pauli_string[i] = 'Z'
                pauli_string[j] = 'Z'
                pauli_strings.append(''.join(pauli_string))
        else:
            if i<n_visible:
                
                pauli_string = ['I'] * num_qubits
                
                pauli_string[i] = 'Z'
                pauli_string[j] = 'Z'
                   
                pauli_strings.append(''.join(pauli_string))       
    return pauli_strings

generate_pauli_strings_tfim(7,3,False)   

['XIIIIII',
 'IXIIIII',
 'IIXIIII',
 'IIIXIII',
 'IIIIXII',
 'IIIIIXI',
 'IIIIIIX',
 'ZIIIIII',
 'IZIIIII',
 'IIZIIII',
 'IIIZIII',
 'IIIIZII',
 'IIIIIZI',
 'IIIIIIZ',
 'ZZIIIII',
 'ZIZIIII',
 'ZIIZIII',
 'ZIIIZII',
 'ZIIIIZI',
 'ZIIIIIZ',
 'IZZIIII',
 'IZIZIII',
 'IZIIZII',
 'IZIIIZI',
 'IZIIIIZ',
 'IIZZIII',
 'IIZIZII',
 'IIZIIZI',
 'IIZIIIZ']

In [3]:
import seaborn as sns

def evaluate_qbm(qbm,testing_dataset,cluster,plot=False,quantile=0.95,method='min'):


    #training_data=numpy.expand_dims(training_data[:,0],axis=1)
    outliers = qbm.get_binary_outliers(
    dataset=testing_dataset, outlier_index=cluster)

    #outliers=numpy.expand_dims(outliers[:,0],axis=1)
    

    points = qbm.get_binary_cluster_points(
    dataset=testing_dataset, cluster_index=cluster-1)

    #points=numpy.expand_dims(points[:,0],axis=1)
    #print(points)
    predict_points_cluster = np.zeros(len(points), dtype=int)
    predict_points_outliers = np.zeros(len(outliers), dtype=int)
    qbm.calculate_outlier_threshold(quantile, method)
    print("Outlier threshold: ", qbm.outlier_threshold)
    print("Calculate outlier Energy")

    testing_data, testing_labels = split_dataset_labels(testing_dataset)
#testing_data=numpy.expand_dims(testing_data[:,0],axis=1)

    outlier_energy = []
    for index, outlier in enumerate(tqdm(outliers), 0):
        outlier = np.reshape(outlier, (qbm.dim_input))
        predict_points_outliers[index], this_outlier_energy = qbm.predict_point_as_outlier(
            outlier,method)
        outlier_energy.append(this_outlier_energy)
    outlier_energy = np.array(outlier_energy)

    o = outlier_energy.reshape((outlier_energy.shape[0]))

 
    print("Calculate cluster energy")
    cluster_point_energy = []

    for index, point in enumerate(tqdm(points), 0):
        point = np.reshape(point, (qbm.dim_input))
        predict_points_cluster[index], this_cluster_point_energy = qbm.predict_point_as_outlier(
        point,method)
        cluster_point_energy.append(this_cluster_point_energy)
    cluster_point_energy = np.array(cluster_point_energy)

    c = cluster_point_energy.reshape((cluster_point_energy.shape[0]))

    title='test'
#qbmqsp.src.utils.save_output(title="cluster_" + title, object=c)
#QBM.plot_energy_diff([o, c], qbm.outlier_threshold, title + ".pdf")

#QBM.plot_hist(c, o, qbm.outlier_threshold, "qbm_hist" + ".pdf")

########## OUTLIER CLASSIFICATION ##########
    print('Outlier classification: Results...')
    predict_points = np.concatenate(
        (predict_points_cluster, predict_points_outliers))

    print("Predicted points test: ", predict_points)

    true_points = np.concatenate(
        (np.zeros_like(cluster_point_energy), np.ones_like(outlier_energy)))

    accuracy, precision, recall = accuracy_score(true_points, predict_points), precision_score(
        true_points, predict_points), recall_score(true_points, predict_points)
    f1 = f1_score(true_points, predict_points)
    tn, fp, fn, tp = confusion_matrix(
        true_points, predict_points, labels=[0, 1]).ravel()
    print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}, \nNum True Negative: {tn}, Num False Negative: {fn}, Num True Positive: {tp}, Num False Positive: {fp}')

#print(f'Wallclock time: {(end-start):.2f} seconds')
    lab=cluster-1
    print("Outlier threshold: ", qbm.outlier_threshold)
    print("Average clusterpoint energy: ", np.average(cluster_point_energy))
    print("Outlier energy: ", outlier_energy)
    
    if plot==True:
        plt.figure()
        plt.title('Test Dataset')
        sns.scatterplot(x=testing_data[:,0],y=testing_data[:,1])
        sns.scatterplot(x=testing_data[:,0][testing_labels>lab],y=testing_data[:,1][testing_labels>lab], c='r',palette='coolwarm')
        
    #plt.title('Predicted Points')
    #sns.scatterplot(x=testing_data[:,0],y=testing_data[:,1], hue=predict_points,palette='coolwarm')
    return [[accuracy],[precision],[recall],[f1],[tn],[fp],[fn],[tp]] 

        

    
    
    
class QBM():
    """Quantum Boltzmann machine (QBM) based on quantum signal processing.

    Parameters
    ----------
    β, enc:
        Same as attributes.
    h, θ:
        See qbmqsp.hamiltonian.Hamiltonian
    δ, polydeg:
        See qbmqsp.qsp_phase_engine.QSPPhaseEngine
    hnodes : Number of hidden nodes
    epochs: Number of epochs to train
    restricted (bool): [default] True
    
    
    Attributes
    ----------
    β : float
        Inverse temperature.
    enc : str in {'general', 'lcu'}
        Unitary block encoding scheme.
    H : qbmqsp.hamiltonian.Hamiltonian
        Constructed from parameters (h, θ).
    qsp : qbmqsp.qsp_phase_engine.QSPPhaseEngine
        Constructed from parameters (δ, polydeg).
    qevt : qbmqsp.qevt.QEVT
        Quantum eigenvalue transform to realize matrix function f(A) = exp(- τ * |A|). Updated after each training epoch.
    observables : qml.operation.Observable
        Observables w.r.t which the QBM is measured to optimize via gradient descent.
    aux_wire, enc_wires, sys_wires, env_wires : list[int]
        Quantum register wires of quantum circuit that prepares and measures the QBM.
    """
    
    
    
    
    
    def __init__(self,data,h: list[str], θ: np.ndarray[float], enc: str, δ: float, polydeg: int, β: float, hnodes,epochs=1,restricted=True,transverse=False,t_bias=None) -> None:
        if β < 0:
            raise ValueError("__init__: β must not be negative.")
        
        self.epochs=epochs
        self.β = β
        self.transverse=transverse
        self.enc = enc
        self.dim_input=n_visible
        self.n_hidden_nodes=hnodes
       
        self.H = Hamiltonian(h, θ)
        self.qsp = QSPPhaseEngine(δ, polydeg)
        self.qevt = self._construct_qevt()
        self.aux_wire, self.enc_wires, self.sys_wires, self.env_wires = self._construct_wires()
        self.observables = self._construct_obervables()
        self.t_bias=t_bias
        
        self.encoded_data, bits_input_vector, num_features = self.binary_encode_data(data, use_folding=True)
        #self.dim_input = bits_input_vector * num_features
        self.quantile=0.95
       
    
        
        self.restricted=restricted

        self.H.θ[0:self.dim_input+self.n_hidden_nodes]=t_bias
        if transverse==True:
           
            if self.restricted:
                self.weights_visible_to_hidden=np.reshape(self.H.θ[2*self.dim_input+2*self.n_hidden_nodes:],(self.dim_input,self.n_hidden_nodes))
                self.biases_hidden=self.H.θ[2*self.dim_input+self.n_hidden_nodes:2*self.dim_input+2*self.n_hidden_nodes]
                self.biases_visible=self.H.θ[self.dim_input+self.n_hidden_nodes:2*self.dim_input+self.n_hidden_nodes]
            else:
                
                self.weights_visible_to_visible,self.weights_visible_to_hidden=self.get_weights(self.H.θ[self.n_hidden_nodes+self.dim_input:])
                self.biases_hidden=self.H.θ[2*self.dim_input+self.n_hidden_nodes:2*self.dim_input+2*self.n_hidden_nodes]
                self.biases_visible=self.H.θ[self.dim_input+self.n_hidden_nodes:2*self.dim_input+self.n_hidden_nodes]
            

    def get_binary_cluster_points(self,dataset, cluster_index: int) -> np.ndarray:
        points = np.array([entry[:-1]
                           for entry in dataset if entry[-1] <= cluster_index])

        return self.binary_encode_data(points, use_folding=False)[0]
    
    def get_binary_outliers(self,dataset, outlier_index: int):
        outliers = np.array([entry[:-1]
                            for entry in dataset if entry[-1] >= outlier_index])

        return self.binary_encode_data(outliers, use_folding=False)[0]
  
    def binary_encode_data(self,data, use_folding=False):
        """ Encode a numpy array of form [[numpy.int64 numpy.int64] ...] into a
        list of form [[int, int, int, ...], ...].
        Example: encode [[107  73] [113  90] ...] to
        [[1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1],[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0] .
        """

        # find out how many bits we need for each feature
        number_bits = len(np.binary_repr(np.amax(data)))
        number_features = data.shape[1]

        binary_encoded = ((data.reshape(-1, 1) & np.array(2 **
                          np.arange(number_bits-1, -1, -1))) != 0).astype(np.float32)
        if use_folding:
            return binary_encoded.reshape(len(data), number_features*number_bits), number_bits, number_features
        else:
            return binary_encoded.reshape(len(data), number_features, number_bits), number_bits, number_features
    
    
    def n_qubits(self, registers: str | set[str] = None) -> int:
        """Return number of qubits per registers.
        
        Parameters
        ----------
        registers : str | set[str]
            Quantum registers whose number of qubits should be returned.
            Must be an element from or a subset of {'aux', 'enc', 'sys', 'env'}.

        Returns
        -------
        n : int
            Number of qubits used per registers.
        """
        if registers is None:
            registers = {'aux', 'enc', 'sys', 'env'}
        elif type(registers) == str:
            registers = {registers}
        if not registers.issubset({'aux', 'enc', 'sys', 'env'}):
            raise ValueError("n_qubits: registers must be an element from or a subset of %r." % {'aux', 'enc', 'sys', 'env'})
        
        n = 0
        if 'env' in registers:
            n += self.qevt.n_qubits('sys')
        registers.discard('env')
        if len(registers) != 0:
            n += self.qevt.n_qubits(registers)
        return n

    def _generate_qsp_phases(self) -> np.ndarray[float]:
        τ = self.β / (1-self.qsp.δ) * self.H.θ_norm()
        φ = self.qsp.generate(τ)
        return φ

    def _construct_qevt(self) -> QEVT:
        φ = self._generate_qsp_phases()
        h_δ, θ_δ = self.H.preprocessing(self.qsp.δ)
        return QEVT(h_δ, θ_δ, self.enc, φ)
    
    def _construct_wires(self) -> tuple[list[int], list[int], list[int], list[int]]:
        wires = list(range(self.n_qubits()))
        aux_wire = wires[: self.n_qubits('aux')]
        enc_wires = wires[self.n_qubits('aux') : self.n_qubits({'aux', 'enc'})]
        sys_wires = wires[self.n_qubits({'aux', 'enc'}) : self.n_qubits({'aux', 'enc', 'sys'})]
        env_wires = wires[self.n_qubits({'aux', 'enc', 'sys'}) : self.n_qubits({'aux', 'enc', 'sys', 'env'})]
        return aux_wire, enc_wires, sys_wires, env_wires

    def _construct_obervables(self) -> list[qml.operation.Observable]:
        n_aux_enc = self.n_qubits({'aux', 'enc'})
        aux_enc_wires = self.aux_wire + self.enc_wires
        proj0 = qml.Projector( [0] * n_aux_enc, aux_enc_wires)

        new_sys_wires = list(range(self.n_qubits('sys')))
        wire_map = dict(zip(self.sys_wires, new_sys_wires))
        observables = [proj0] + [proj0 @ string_to_pauli_word(self.H.h[i], wire_map) for i in range(self.H.n_params)]
       
            
        return observables
    
    def _bell_circuit(self) -> None:
        for i, j in zip(self.sys_wires, self.env_wires):
            qml.Hadamard(i)
            qml.CNOT([i, j])

    def probabilistic(self):
        
        bit_strings=[]
        for i in range(2**(self.n_hidden_nodes+self.dim_input)):
        # Convert the number to its binary representation and pad with leading zeros
            bit_string = bin(i)[2:].zfill(self.n_hidden_nodes+self.dim_input)
             
            bit_list = np.array([int(bit) for bit in bit_string])
            bit_strings.append(bit_list) 
            
      
        sample = random.choices(bit_strings, k=1)

        for i,x in enumerate(sample[0]):
            if x==1:
                qml.PauliX(wires=[self.sys_wires[i]])
    
    def _prepare(self) -> None:
        self._bell_circuit()
        #self.probabilistic()
        self.qevt.circuit(self.aux_wire, self.enc_wires, self.sys_wires)
    
    def _measure(self) -> None:
        #return qml.sample(wires=self.aux_wire+self.enc_wires+self.sys_wires)
        
        return [qml.expval(self.observables[i]) for i in range(len(self.observables))]
    
    
    def get_sample(self,shots=1):
        dev = qml.device(dev_name,shots=shots, wires=self.n_qubits({'aux','enc','sys'}))
        @qml.qnode(dev)
        
        def quantum_circuit():
            
            self._prepare()
            return qml.sample(wires=self.aux_wire+self.enc_wires+self.sys_wires)
        
        sample=quantum_circuit()
        return sample
    
    def get_average_configuration_from_samples(self, samples: list, input_vector=None):
        ''' Takes samples from Annealer and averages for each neuron and connection
        '''

        # unclamped if input_vector == None
        unclamped = input_vector== None
        
        # biases (row = sample, column = neuron)
        np_samples = np.vstack(
            tuple([np.array(list(sample.values())) for sample in samples]))
        avgs_biases = np.average(np_samples, axis=0)
        avgs_biases_hidden = avgs_biases[self.dim_input:] if unclamped else avgs_biases
        avgs_biases_visible = avgs_biases[:
                                          self.dim_input] if unclamped else input_vector

        # weights
        avgs_weights_visible_to_hidden = np.zeros(
            self.weights_visible_to_hidden.shape)
        if not self.restricted:
            avgs_weights_visible_to_visible = np.zeros(
                self.weights_visible_to_visible.shape)
        for v in range(self.dim_input):
            # visible to hidden connections
            for h in range(self.n_hidden_nodes):
                x, y = (np_samples[:, v], self.dim_input +
                        h) if unclamped else (input_vector[v], h)
                avgs_weights_visible_to_hidden[v, h] = np.average(
                    x*np_samples[:, y])
            # visible to visible connections
            if not self.restricted:
                for v2 in range(v, self.dim_input):
                    x, y = (np_samples[:, v], np_samples[:, v2]) if unclamped else (
                        input_vector[v], input_vector[v2])
                    avgs_weights_visible_to_visible[v, v2] = np.average(x*y)

        if self.restricted:
            return avgs_biases_hidden, avgs_biases_visible, avgs_weights_visible_to_hidden, None
        else:
            return avgs_biases_hidden, avgs_biases_visible, avgs_weights_visible_to_hidden, avgs_weights_visible_to_visible

    def _compute_expvals(self) -> np.ndarray[float]:
        dev = qml.device(dev_name,wires=self.n_qubits({'aux','enc','sys','env'}))
        #dev = qml.device(dev_name, backend=backend,wires=self.n_qubits(),ibmqx_token=token)
        @qml.qnode(dev)
        
        def quantum_circuit():
            self._prepare()
            if dev_name=='default.qubit':
                return self._measure()
            else:
                observation_wires=[[]]
                
                for term in self.H.h:
                    list=[]
                    for i,el in enumerate(term):
                        if el!='I':
                            list.append(i+self.n_qubits({'aux','enc'}))
                    observation_wires.append(list)
                
                probabilities=[qml.probs(wires=[0,1]+wires) for wires in observation_wires]

                return probabilities
                
            
        
        if dev_name=='default.qubit':
        
            measurements = quantum_circuit()
            
            success_probability = measurements[0]
           
            
            qbm_expvals = measurements[1:] / success_probability
            
            return qbm_expvals
        
        else:

            probabilities=quantum_circuit()
            success_probability=probabilities[0][0]
            print(success_probability)
            rest_probabilites=probabilities[1:]
            exp_val=[]
            for i,term in enumerate(rest_probabilites):
                if i<self.n_hidden_nodes+self.dim_input:
                
                    ps=np.reshape(term,(4,2))[0]
                
                    exp_val.append(ps[0]-ps[1])
                else:
                
                    ps=np.reshape(term,(4,4))[0]
        
                    exp_val.append(ps[0]-ps[1]-ps[2]+ps[3])      
            
        
        
       
            return np.array(exp_val)/success_probability
    
    def _loss_func(self, ρ0: np.ndarray[float], ρ1: np.ndarray[float]) -> float:
        return relative_entropy(ρ0, ρ1, check_state=True).item()
    
    def assemble(self) -> np.ndarray[float]:
        """Assemble QBM."""
        expH = spl.expm(-self.β * self.H.assemble())
        return expH / np.trace(expH)
    
    
    def get_energy(self,input_vector,k=10,method='min'):
        input_vector=[input_vector]
        new_biases=self.biases_hidden+np.matmul(1-2*np.array(input_vector),self.weights_visible_to_hidden).flatten()
      

        D=new_biases**2+self.t_bias[self.dim_input:]**2
        
        exp_vals=-(new_biases/D)*np.tanh(self.β*D)

        x_vals=-(self.t_bias[self.dim_input:]/D)*np.tanh(self.β*D)
        print(exp_vals,x_vals)
        z_energy= exp_vals*self.H.θ[2*self.dim_input+self.n_hidden_nodes:2*self.dim_input+2*self.n_hidden_nodes]
        x_energy= x_vals*self.t_bias[self.dim_input:]
        
        return z_energy+x_energy

            
     
    
    def free_energy(self,method='min',input_vector=None):
        '''Function to compute the free energy'''

        # calculate hidden term
        
         
        if self.n_hidden_nodes==0: 
            hidden_term=0
        else:
            hidden_term = self.get_energy(method=method,input_vector=input_vector)

        # calculate visible_term
        # visible bias
        visible_term = np.dot(
            1-2*np.array(input_vector), self.H.θ[self.dim_input+self.n_hidden_nodes:2*self.dim_input+self.n_hidden_nodes]) #/beta
        
        pos_neg=1-2*input_vector
        if self.restricted==False:
             
             
             vv_interaction=np.matmul(self.weights_visible_to_visible,pos_neg)
             vv_interaction=np.matmul(pos_neg.T,vv_interaction)
             visible_term=visible_term+vv_interaction
        

        return hidden_term + visible_term
    
    def calculate_outlier_threshold(self, quantile=0.95,method='min'):
        
        self.quantile = quantile
        energy_func=partial(self.free_energy,method)
        
        energies = np.apply_along_axis(
            energy_func, axis=1, arr=self.encoded_data)
        
        self.outlier_threshold = np.quantile(energies, self.quantile)
        
        
    
    
    
    
    def get_average_configurations(self,input_vector=None):
        '''
        Function for giving averge configurations of all qubits for the gibbs state of the system.
        
        If input vector is clamped at a certain value , it gives configuration of hidden units only.
       
    
        Parameters:
        input vector (np.ndarray)
        
    
        Returns:
        list: List of expectation values of hamilatonian terms.
        '''
        
        
        # unclamped values
        if input_vector is None:
            
            qbm_expvals=self._compute_expvals()
            
            return qbm_expvals
        
        # clamped values
        
        
        if self.restricted:
            self.weights_visible_to_hidden=np.reshape(self.H.θ[self.dim_input+self.n_hidden_nodes:],(self.dim_input,self.n_hidden_nodes))
            self.biases_hidden=self.H.θ[self.dim_input:self.dim_input+self.n_hidden_nodes]
            self.biases_visible=self.H.θ[:self.dim_input]
        else:
            
            self.weights_visible_to_visible,self.weights_visible_to_hidden=self.get_weights(self.H.θ[self.n_hidden_nodes+self.dim_input:])
            self.biases_hidden=self.H.θ[2*self.dim_input+self.n_hidden_nodes:2*self.dim_input+2*self.n_hidden_nodes]
            self.biases_visible=self.H.θ[self.dim_input+self.n_hidden_nodes:2*self.dim_input+self.n_hidden_nodes]
        
        input_vector=[input_vector]
        new_biases=self.biases_hidden+np.matmul(1-2*np.array(input_vector),self.weights_visible_to_hidden).flatten()
        #np.matmul(input_vector, self.weights_visible_to_hidden).flatten()
        
        Q_new=new_biases

        D=new_biases**2+self.t_bias[self.dim_input:self.dim_input+self.n_hidden_nodes]**2
        
        exp_vals=-(new_biases/D)*np.tanh(self.β*D)
        
        return exp_vals
            
       
        
        
    def train_for_one_iteration(self, batch, learning_rate):

        '''
        Performs the update for one batch in the dataset.

        Parameters:

        batch 
        learning_rate
       
        Returns:
        list: List of avg errors in the visible configuration for each element of the batch.
        '''
        
        
        errors = 0
        #errors_biases_visible = 0
        #errors_weights_visible_to_hidden = 0
        #if not self.restricted:
          #  errors_weights_visible_to_visible = 0

        for i,input_vector in enumerate(batch):
            
            
            if i==0:
                unclamped_config = self.get_average_configurations() 
                
                if self.transverse==True:
                    unclamped_config=unclamped_config[self.n_hidden_nodes+self.dim_input:]
                
                #print(unclamped_config)
            
            clamped_config = self.get_average_configurations(input_vector) # has only expectations over hidden units
            
            # avgs_weights_visible_to_visible_clamped only has a value if not restricted
            #print(clamped_config)
            
            # Getting averages for all qubits , avg_visible=input_vector
        
            
            
            full_clamped_config=np.zeros_like(unclamped_config)
            
            full_clamped_config[:self.dim_input]=1+(-2)*input_vector   
            full_clamped_config[self.dim_input:self.dim_input+self.n_hidden_nodes]=clamped_config
            
            pos_neg=1-2*input_vector
            if self.restricted:
                
                
                full_clamped_config[self.dim_input+self.n_hidden_nodes:]=np.kron(pos_neg,clamped_config)
            
            else:
                
                
                
                visible=list(pos_neg[j]*pos_neg[i] for j in range(len(pos_neg)) for i in range(j+1,len(pos_neg)))
                hidden=np.kron(pos_neg,clamped_config)
                for i in range(1,self.dim_input+1):
                    for j in range(self.n_hidden_nodes):
                        visible.insert((i-1)*self.n_hidden_nodes+(i)*(self.dim_input-1)-(i-1)+j,hidden[self.n_hidden_nodes*(i-1)+j])
                full_clamped_config[self.dim_input+self.n_hidden_nodes:]=np.array(visible)
            
            errors += full_clamped_config - unclamped_config
            
            
            
            
            
            
            

        errors /= batch.shape[0]
        
        self.H.θ[self.dim_input+self.n_hidden_nodes:] = self.H.θ[self.dim_input+self.n_hidden_nodes] - learning_rate * errors
                
        self.qevt = self._construct_qevt()
                
       
        
        
        return np.average(errors[:self.dim_input]**2)
    
    
    
    
    def train_model(self, batch_size=8, learning_rate=0.005,save=False):
        
        data = self.encoded_data
        
        weights=[]
        batch_num = data.shape[0] // batch_size
        diff = data.shape[0] % batch_size
        self.batch_size=batch_size
        if diff:
            
        
            data = data[:-diff]
            last_batch = data[data.shape[0] - diff:]
        
        
        
        batches = np.vsplit(data, batch_num)
        
        if diff:
            batches.append(last_batch)
              
        losses=[]
        
        for epoch in range(1, self.epochs+1):
            print(f'Epoch {epoch}')
            batch_errors = None
            batchnum = 1
            errors_epoch=[]
            for batch in tqdm(batches):
                    #print(batch)
                    errors = self.train_for_one_iteration(batch, learning_rate)
                    
                    if type(batch_errors) is np.ndarray:
                        batch_errors = np.hstack((batch_errors, errors))
                    else:
                        batch_errors = errors
                    #self.save_weights(
                        #f'e{epoch}_b{batchnum}_{self.paramstring}')
                    batchnum += 1
               
                    #self.save_weights(
                     #   f'e{epoch}_b{batchnum}_{self.paramstring}')
                    #raise e
                    errors_epoch.append(errors)
            
            losses.append(errors_epoch)
            weights.append(self.H.θ)
            if save==True:
                try:
                    np.savez(f'./epoch{epoch}_weights_h{self.n_hidden_nodes}_v{self.dim_input}_lr{self.learning_rate}_e{self.epochs}',self.H.θ)
                    np.savez(f'./epoch{epoch}_losses_h{self.n_hidden_nodes}_v{self.dim_input}_lr{self.learning_rate}_e{self.epochs}',errors_epoch)
                except:
                    print('error_saving')
        self.calculate_outlier_threshold(self.quantile)
        
        
        
        return losses, weights 
    
    #self.error_container.add_error(batch_errors)
        #self.error_container.plot("qbm_errors" + self.paramstring)
        #self.save_weights(title="final_weights_qbm" + self.paramstring)
        # make list of values of the error dicts
        
        #self.calculate_outlier_threshold(self.quantile)
       
    def predict_point_as_outlier(self, input,method):
        energy = self.free_energy(method,input)
        if energy >= self.outlier_threshold:
            return 1, energy
        return 0, energy
        
    
    def get_weights(self,Q):
        weights_vh_vv=list(Q[self.dim_input+self.n_hidden_nodes:])
            
        for i in range(1,self.dim_input+1):
            for j in range(i):
                weights_vh_vv.insert((self.dim_input+self.n_hidden_nodes)*(i-1)+j,0)
            
        weights_vh_vv=np.array(weights_vh_vv)
        weights_visible_to_visible=weights_vh_vv.reshape(self.dim_input,self.dim_input+self.n_hidden_nodes)[:,0:self.dim_input]
        weights_visible_to_hidden=weights_vh_vv.reshape(self.dim_input,self.dim_input+self.n_hidden_nodes)[:,self.dim_input:]
    
        return weights_visible_to_visible,weights_visible_to_hidden
    
    def save_model(path,dataset_name):
         path=Path(path/dataset)
         path.mkdir(exist_ok=True)
         np.savez(f'_e{qbm.epochs}_h{qbm.n_hidden_nodes}_v{qbm.dim_input}_b{qbm.batch_size}',qbm.H.θ)
        
     
    

    

In [4]:
def perform_run(seed=77, n_hidden_nodes=1, sample_count=20,
        beta_eff=1, epochs=2, batch_size=9, learning_rate=0.005,
         restricted=True, save_path=None,t=0, name=""):

    '''
    Performs training and testing of the QBM for one set of parameters.
    '''    
    
    global dev_name
    global data
    global training_data
    global testing_data
    global training_dataset
    global testing_dataset
    
    print("Start")


    
    random.seed(seed)
    np.random.seed(seed)
    

    data = import_dataset(PATH)
    training_dataset, testing_dataset = split_data(data, CLUSTER)
    training_data, training_labels = split_dataset_labels(training_dataset)
    testing_data, testing_labels = split_dataset_labels(testing_dataset)
    
  
    
    print("Seed is " + str(seed))

    start = time.time()

    
    param_string = "_se" + str(seed) + "_h" + str(n_hidden_nodes) + "_b" + str(
        beta_eff) + "_e" + str(epochs) + "_l" + str(learning_rate) + "_r" + str(restricted) + "_n_" + name

    # create DQBM
    n_visible=8
    n_qubits = n_visible+n_hidden_nodes
    multiple=False
    restricted=False
    transverse=True
    h = generate_pauli_strings_tfim(n_qubits,n_visible,restricted=restricted,transverse=transverse)
    nparams=len(h)

    θ_init =np.random.rand(nparams)/nparams#np.loadtxt('./weights_7_3_un.txt')
    enc = 'general'
    δ = 0.3
    polydeg = 10
    
    t_bias=np.zeros(n_qubits)
    t_bias[n_visible:+n_visible+n_hidden_nodes]=t
    
    dev_name='default.qubit'


    qbm = QBM(training_data,h, θ_init, enc, δ, polydeg,beta_eff,n_hidden_nodes,epochs,restricted,transverse=transverse,t_bias=t_bias)
    
    print('Training QBM...')
    errors,weights=qbm.train_model(batch_size,learning_rate)

    
    
  
    
    final_metrics=evaluate_qbm(qbm,testing_dataset,CLUSTER,plot=False,quantile=0.95,method='mean')
    
    if save_path is not None:
        
        try:
       
            path=Path(save_path)/param_string
            path.mkdir(exist_ok=True)
            np.save('errors.npy',errors)
            np.save('weights.npy',weights)
            np.save('training_dataset.npy',training_dataset)
            np.save('metrics.npy',np.array(final_metrics))
        except:
            print('Error Saving')
    
    return final_metrics


    

In [5]:
def get_averages(list_of_lists):
    array_of_arrays = np.array(list_of_lists)
    averages = np.mean(array_of_arrays, axis=0)
    return averages



def configure_hyperparams(run):
    '''
    Sets the hyperparameters for a given run. Both with
    and without hyperparameter optimisation.
    '''
    
    global BATCH_SIZE
    global EPOCHS
    
    global T_BIAS
    global N_HIDDEN_UNITS
    global SAMPLE_COUNT
    global BETA_EFF
    global LEARNING_RATE
    #global BASEPATH_RECORD
    global RESTRICTED
    global PARAM_STRING
    #global ARGUMENTS

    if run:
        
        config_defaults = {'batchsize': args.batchsize, 'epochs': args.epochs,'hnodes': args.hnodes,
                            'beta_eff': args.beta_eff, 't_bias':0.01,
                           'restricted':args.restricted, 'learning_rate':args.learning_rate
                           }
    
        run.config.setdefaults(config_defaults)

        # set hyperparameters from sweep
        BATCH_SIZE = wandb.config.batchsize
        EPOCHS = wandb.config.epochs
        LEARNING_RATE=wandb.config.learning_rate
        # define network parameters for dqbm
        N_HIDDEN_UNITS = wandb.config.hnodes
        T_BIAS=wandb.config.t_bias
        BETA_EFF = wandb.config.beta_eff
        
        RESTRICTED=wandb.config.restricted
       
    else:
        # set hyperparameters
        BATCH_SIZE = args.batchsize
        EPOCHS = args.epochs
        LEARNING_RATE=args.learning_rate

        # define network parameters for dqbm
        N_HIDDEN_UNITS = args.hnodes
        T_BIAS=args.t_bias
        BETA_EFF = args.beta_eff
        RESTRICTED=args.restricted
        
     # params and identifier-strings independent of using hyperparameter
    # optimization or not
    #BASEPATH_RECORD = get_basepath("training", DATASET)
    
    #ARGUMENTS = [BATCH_SIZE, EPOCHS, ACCURACY_INTERVAL,
             #   N_HIDDEN_UNITS, SAMPLE_COUNT, ANNEAL_STEPS, BETA_EFF, TESTSIZE]
    
    PARAM_STRING = "_h" + str(N_HIDDEN_UNITS) +"_ba"+str(BATCH_SIZE) +"_b" + str(
        BETA_EFF)+"_lr"+str(LEARNING_RATE)+'_t'+str(T_BIAS) + "_e" + str(EPOCHS) + "_r" + str(RESTRICTED)
    
    
    return PARAM_STRING










def main(args):

    
  
   
    

    NUM_RUNS = args.n_runs

    # start run
    if HYPERPARAM_OPT:
        run = wandb.init(reinit=True
                         , group=SWEEP_ID
                         )
    else:
        run = None

    param_string_for_run=configure_hyperparams(run)
    
    #is_first_run = args.first_run

    PARAM_STRING_SEED=param_string_for_run
    


    
    if HYPERPARAM_OPT:
        
        
        
        num_metrics=8   #number of metric returned by the perform_run function
        metrics_for_all_seeds=[[] for i in range(num_metrics)]
    
    seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 14, 15, 16, 17, 18, 19, 21, 22,
             23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
             40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 54, 55, 56, 58,
             60, 61, 62, 63, 64, 65, 66, 70, 71, 72, 73, 74, 75, 76, 77, 78,
             79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 92, 93, 96, 97, 98, 99]

    NUM_RUNS=4
    
    for run_num in range(NUM_RUNS):
        # determine seed
        seed = random.choice(seeds)
        
        print("seed: " + str(seed))
        
        PARAM_STRING_SEED = PARAM_STRING_SEED+ "_s" + str(seed)
    
        #files_for_this_run = create_or_find_files(is_first_run, seed)
        #is_first_run = 'False'
    
        
        metrics=perform_run(epochs=EPOCHS, n_hidden_nodes=N_HIDDEN_UNITS    ,
            batch_size=BATCH_SIZE,beta_eff=BETA_EFF,
            learning_rate=LEARNING_RATE, t=T_BIAS,restricted=RESTRICTED,
            seed=seed,save_path=args.save_path)
        
      

        if HYPERPARAM_OPT:
            # get combined metric of accuracy and auc-roc-score
            #combined_metrics = get_combined_metrics(metrics[0], metrics[5])
            #metrics.append(combined_metrics)
            
             for metric_index in range(len(metrics)):
                metrics_for_all_seeds[metric_index].append(metrics[metric_index])

        print(f"Training for seed {seed} done.\n")
        seeds.remove(seed)
    
    
    if HYPERPARAM_OPT:
        #print(metrics_for_all_seeds)
        
        run.name = PARAM_STRING_SEED
        
        print(f"Run Name: {run.name}\n")
        for metric_index in range(len(metrics_for_all_seeds)):
            metrics_for_all_seeds[metric_index] = get_averages(metrics_for_all_seeds[metric_index])
        #print(metrics_for_all_seeds)
        
        
        #for i in range(len(metrics_for_all_seeds[0])):
        wandb.log({
                       'accuracy': metrics_for_all_seeds[0].item(),
                   'precision': metrics_for_all_seeds[1].item(), 'recall': metrics_for_all_seeds[2].item(),
                   'f1_score': metrics_for_all_seeds[3].item(), 'true_negatives': metrics_for_all_seeds[4].item(),
                   'false_positives': metrics_for_all_seeds[5].item(),
                       'false_negatives': metrics_for_all_seeds[6].item(),
                   'true_positives': metrics_for_all_seeds[7].item()}
                  )
        run.finish()
    print("Run done.")

#print("Seeds from list left: ", seeds)
         






In [ ]:
import argparse
import wandb
CLUSTER=5
PATH= '../../datasets/good_datasets/l_o8_c5_d2_v0.35_p190_4.npy'


    




name = 'sweep_' + 'transverse-1' 

sweep_configuration = {'name': name,
                       'description': '',
                       'project': "qbm-anomaly-detection-transverse", 'entity': "shivang_arora",
                       'method': 'random',
                       'metric': {'goal': 'maximize', 'name':
                           'f1_score'},
                       'parameters': {'batchsize': {'value':73},
                                      'epochs': {'value':7},
                                      'learning_rate': {'value': 0.6058642443189942},
                                      'beta_eff':{'value': 1.7782274212954747},
                                       't_bias':{'max':5.0,'min': -5.0},
                                      'hnodes': {'value':1},
                                      },
                        
                       # min_iter same number as accuracy-interval?
                       #'early_terminate': {'type': 'hyperband', 'min_iter': 4}
                       
                       }






if __name__ == '__main__':
    

    
    
    parser = argparse.ArgumentParser(
        description='Generate clustered datasets with outliers.')
   
    

    
        
    parser.add_argument('-hn', '--hnodes',
                        metavar='INT',
                        help='Amount of hidden units for RBM model',
                        default=1,  # best fit for dataset l_o7_c5_d3_p200_v1
                        type=int)

    parser.add_argument('-e', '--epochs',
                        metavar='INT',
                        help='Epochs for training',
                        default=2,  # best fit for dataset l_o7_c5_d3_p200_v1
                        type=int)

    parser.add_argument('-b', '--batchsize',
                        metavar='INT',
                        help='Batchsize for training',
                        default=9,  # best fit for dataset l_o7_c5_d3_p200_v1
                        type=int)

    
   
    # random seed for reproducibility of classical (SA) runs
    
    parser.add_argument('-s', '--seed',
                        metavar='INT',
                        help='Seed for RNG',
                        default=77,  # best fit for dataset l_o7_c5_d3_p200_v1
                        type=int)

 

    parser.add_argument('--save_path',
                        help='Filepath to save',
                        default=None,
                        type=str)

    parser.add_argument('--name',
                        help='Name for run',
                        default="",
                        type=str)

    
    parser.add_argument('--t_bias',
                        help='Name for run',
                        default=0.01,
                        type=float)
    #parser.add_argument("--sample_count", type=int, default=20)
   
    # number of annealing steps to perform (only relevant for SA)
    
 
   
    # inverse effective temperature, only relevant for runs on D-Wave
    
    # (What is good default value? 0.5? 1.0? ...?)
    parser.add_argument("--beta_eff", type=float, default=1.0)

   

    parser.add_argument("--learning_rate", type=float, default=0.005)
    
    parser.add_argument("--restricted", type=str, default='false')

  
    
    parser.add_argument("--n_runs", type=int, default=2)

    # for automized hyperparameter optimization
    
    parser.add_argument("--hyperparam_opt", type=str, default='true')
    parser.add_argument("--n_sweeps", type=int, default=500)
    parser.add_argument("--sweep_id", type=str, default=None)
    
    #Add sweep id , through a separate sweep file instead.
    
    parser.add_argument("--key", type=str, default=None)
    
    args = parser.parse_args([])
    
    
    HYPERPARAM_OPT = args.hyperparam_opt == 'true'
    NUM_SWEEPS = args.n_sweeps

    
    
    if HYPERPARAM_OPT:
        #wandb.login(key=args.key)
        wandb.login()
        
        #SWEEP_ID = args.sweep_id
        
        #sweep_id_path = "mdsg/qbm-anomaly-detection/" + SWEEP_ID
        n_visible=8
        project_name='Gate-qbm-anomaly-detection-transverse' 
        entity='shivang_arora'
        # add entity name
        sweep_id_path=wandb.sweep(sweep_configuration,project=project_name,entity=entity)    
        
        sweep_id_path=project_name+'/'+sweep_id_path
        print(sweep_id_path)
        SWEEP_ID=sweep_id_path
        main_with_args = partial(main, args)
        wandb.agent(sweep_id=sweep_id_path, function=main_with_args,
                    count=NUM_SWEEPS)
    else:
        main(args)



wandb: Currently logged in as: shivang-arora (shivang_arora). Use `wandb login --relogin` to force relogin


Create sweep with ID: iok1td1z
Sweep URL: https://wandb.ai/shivang_arora/Gate-qbm-anomaly-detection-transverse/sweeps/iok1td1z
Gate-qbm-anomaly-detection-transverse/iok1td1z


wandb: Agent Starting Run: emokx9hq with config:
wandb: 	batchsize: 73
wandb: 	beta_eff: 1.7782274212954747
wandb: 	epochs: 7
wandb: 	hnodes: 1
wandb: 	learning_rate: 0.6058642443189942
wandb: 	t_bias: 1.428721816043364


seed: 70
Start
Seed is 70

Starting MATLAB engine.. Done.

Training QBM...
Epoch 1


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:28<00:00,  4.11s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:47<00:00,  6.76s/it]


Epoch 6


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:56<00:00,  8.03s/it]


Epoch 7


 71%|████████████████████████████████████████████████████████████                        | 5/7 [00:38<00:15,  7.63s/it]